# Building MathSolving Problem with agentlite
This tutorial is a simple case on how to design a math solving agent with agentlite and WolframAlpha. Designing the math solving agent contains three steps:

- Defining the WolframAlphaSolver action with Wolframalpha api

- Defining the MathSolveAgent with WolframAlphaSolver action

- Test the MathSolveAgent

## Define WolframAlphaSolver action
For WolframAlphaSolver action, we need to provide three properties following the design of BaseAction.

- action_name

- action_desc

- params_doc

Agent will use these three property to understand how to use this action.
And the action execution is implemented in __call__() method. The params_doc is a dictionary to explain the input parameters in __call__() method.

You need to get your own api_keys from the corresponding website:

- openai_api_key: https://openai.com/blog/openai-api

- wolfram_alpha_api_key: https://developer.wolframalpha.com/access

In [1]:
import os
from agentlite.actions.BaseAction import BaseAction
import wolframalpha
from typing import Tuple
from env import openai_api_key
from env import wolfram_alpha_api_key
from agentlite.commons import AgentAct, TaskPackage
from agentlite.actions import ThinkAct, FinishAct
from agentlite.actions.InnerActions import INNER_ACT_KEY



# Define WolframAlphaSolver Action
class WolframAlphaSolver(BaseAction):
    def __init__(self, app_id) -> None:
        action_name = "WolframAlphaSolver"
        action_desc = "Using this action to solve a math problem with WolframAlpha."
        params_doc = {"query": "the math equation to be solved. be simple"}
        self.client = wolframalpha.Client(app_id)
        super().__init__(
            action_name=action_name,
            action_desc=action_desc,
            params_doc=params_doc,
        )
        
    def run(self, query: str) -> Tuple[str, bool]:
        """Run query through WolframAlpha and parse result."""
        from urllib.error import HTTPError

        is_success = False  

        res = self.client.query(query)

        if not res["@success"]:
            return (
                "Your Wolfram query is invalid. Please try a new query for wolfram.",
                is_success,
            )
        assumption = next(res.pods).text
        answer = ""
        for result in res["pod"]:
            if result["@title"] == "Solution":
                answer = result["subpod"]["plaintext"]
            if result["@title"] == "Results" or result["@title"] == "Solutions":
                for i, sub in enumerate(result["subpod"]):
                    answer += f"ans {i}: " + sub["plaintext"] + "\n"
                break
        if answer == "":
            answer = next(res.results).text


        if answer is None or answer == "":
            # We don't want to return the assumption alone if answer is empty
            return "No good Wolfram Alpha Result was found", is_success
        is_success = True
        return f"Assumption: {assumption} \nAnswer: {answer}", is_success

    def __call__(self, query):
        res = self.run(query)
        return res[0]


## Define the math solve agent
For the MathSolveAgent, we need to define several properties:

- actions: A list of available actions (WolframAlphaSolver) for MathSolveAgent

- name: Name of the agent

- role: The agent's role

In [2]:
# defining the MathSolve agent as an agent to use the WolframAlphaSolver action
from typing import List
from agentlite.actions import BaseAction
from agentlite.agents import ABCAgent, BaseAgent
from agentlite.llm.agent_llms import BaseLLM, get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.logging.multi_agent_log import AgentLogger

## using the agentrool logger to record the running log
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False)

# Define the MathSolve agent
class MathSolveAgent(BaseAgent):
    def __init__(
        self,
        llm: BaseLLM,
        actions: List[BaseAction] = [WolframAlphaSolver(wolfram_alpha_api_key)], # we add both actions into agents
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "MathSolve_agent"
        role = """You can answer math questions by WolframAlphaSolver action. Finish it if you find the answer."""
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
            logger=agent_logger,
        )

## Initialize the MathSolveAgent need three steps

- Initialize the backend LLM

- Initialize the agent with LLM

- Add an illustration example for the agent (Optional, buy highly recommended to stabilize the agent)

In [3]:
# Initialize the backend LLM
llm_config_dict = {"llm_name": "gpt-4-32k", "temperature": 0.0, "openai_api_key": openai_api_key}
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)

# Initialize the agent
wolf_action = WolframAlphaSolver(wolfram_alpha_api_key) 
mathsolve_agent = MathSolveAgent(llm=llm, actions = [wolf_action])


# Add an illustration example
exp_task = "What is the result of 1*98+23*45+87*45"
exp_task_pack = TaskPackage(instruction=exp_task)

act_1 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""The math equation is "1*98+23*45+87*45". And I can solve it with WolframAlphaSolver action."""
    },
)
obs_1 = "ok"

act_2 = AgentAct(
    name=wolf_action.action_name,
    params={"query": "1*98+23*45+87*45",
        },
)
obs_2 = """5048"""

act_3 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: "The result is 5048."})
obs_3 = "Task Completed."
exp_act_obs = [(act_1, obs_1), (act_2, obs_2), (act_3, obs_3)]

mathsolve_agent.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)

## Test the agent on other math problems

In [4]:
# calling the agent with TaskPackage
from agentlite.commons import TaskPackage
test_task = "What is the result of 76*23+87/2+45"


test_task_pack = TaskPackage(instruction=test_task)
response = mathsolve_agent(test_task_pack)
print("response:", response)

Agent MathSolve_agent receives the following TaskPackage:
[
	Task ID: 563daf80-6c85-403c-8d40-38d91a2395d9
	Instruction: What is the result of 76*23+87/2+45
]
====MathSolve_agent starts execution on TaskPackage 563daf80-6c85-403c-8d40-38d91a2395d9====


/Users/liangwei.yang/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Agent MathSolve_agent takes 0-step Action:
{
	name: Think
	params: {'response': 'The math equation is "76*23+87/2+45". And I can solve it with WolframAlphaSolver action.'}
}
Observation: OK
Agent MathSolve_agent takes 1-step Action:
{
	name: WolframAlphaSolver
	params: {'query': '76*23+87/2+45'}
}
Observation: Assumption: 76×23 + 87/2 + 45 
Answer: 3673/2
Agent MathSolve_agent takes 2-step Action:
{
	name: Finish
	params: {'response': 'The result is 3673/2.'}
}
Observation: Task Completed.
=========MathSolve_agent finish execution. TaskPackage[ID:563daf80-6c85-403c-8d40-38d91a2395d9] status:
[
	completion: completed
	answer: The result is 3673/2.
]
response: The result is 3673/2.
